In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast

In [ ]:
RUN_ID = 80747

df_conditions = pd.read_csv(f'./results/{RUN_ID}/conditions.csv')

new_conditions_cols = []
for col in df_conditions.columns:
    if col != 'generation': # Don't try to parse 'generation'
        try:
            # Safely evaluate string to tuple
            new_conditions_cols.append(ast.literal_eval(col))
        except (ValueError, SyntaxError):
            new_conditions_cols.append(col) # Keep as is if not a parsable tuple string
    else:
        new_conditions_cols.append(col)
df_conditions.columns = new_conditions_cols

df_conditions_melted = df_conditions.melt(id_vars='generation', var_name='condition', value_name='value')


df_actions = pd.read_csv(f'./results/{RUN_ID}/actions.csv')
df_actions_melted = df_actions.melt(id_vars='generation', var_name='action', value_name='value')

df_performance = pd.read_csv(f'./results/{RUN_ID}/performance.csv')
df_performance_melted = df_performance.melt(id_vars='generation', var_name='metric', value_name='value')

In [ ]:
def format_condition_name(condition_tuple_or_str):
    """
    Formats a condition name from a tuple ('NAME', True/False) or a string.
    If True, prepends 'NOT'.
    """
    if isinstance(condition_tuple_or_str, tuple) and len(condition_tuple_or_str) == 2:
        name, is_negated = condition_tuple_or_str
        if is_negated:
            return f"NOT {name}"
        else:
            return name
    return str(condition_tuple_or_str) # Return as string if it's not a tuple

fig1, ax1 = plt.subplots(figsize=(15, 6))

final_generation_cond = df_conditions['generation'].max()

# Get the conditions for the final generation, sorted by their 'value'
final_generation_conditions_values = df_conditions[df_conditions['generation'] == final_generation_cond].iloc[0].drop('generation')

# Order conditions based on their values in the final generation, from most to least
# This list will still contain the original tuple names for correct categorical ordering
ordered_conditions = final_generation_conditions_values.sort_values(ascending=False).index.tolist()

# Convert 'condition' column to a categorical type with the desired order
# This assumes df_conditions_melted['condition'] might contain string representations of tuples
# The categorical conversion ensures the lines are drawn in the correct order.
df_conditions_melted['condition'] = pd.Categorical(df_conditions_melted['condition'], categories=ordered_conditions, ordered=True)

# Sort the DataFrame by the 'condition' column to ensure correct plotting order for legend
df_conditions_melted_sorted = df_conditions_melted.sort_values(by='condition')

sns.lineplot(data=df_conditions_melted_sorted, x='generation', y='value', hue='condition', ax=ax1)
ax1.set_title("Condition Types Over Generations")
ax1.set_xlabel("Generation")
ax1.set_ylabel("Value")

# Get current legend handles and labels
handles, labels = ax1.get_legend_handles_labels()

# Create new labels with formatted names and final generation values
new_labels = []
for label in labels:
    # Attempt to parse the label into a tuple if it's a string,
    # as legend labels might sometimes be strings even if the underlying data is tuple-like.
    processed_label = label
    if isinstance(label, str):
        try:
            processed_label = ast.literal_eval(label)
        except (ValueError, SyntaxError):
            # If it cannot be parsed, use the original string label
            pass
    
    formatted_label = format_condition_name(processed_label) 
    
    # Use the processed_label (which is now either a tuple or a direct string) to get the value
    value_at_final = final_generation_conditions_values.get(processed_label, 'N/A')
    
    # Check if value_at_final is 'N/A' (a string) before trying to format it as a number
    if value_at_final == 'N/A':
        new_labels.append(f"{formatted_label} ({value_at_final})")
    else:
        new_labels.append(f"{formatted_label} ({value_at_final:.0f})") 

# Update the legend with new labels and correct position
ax1.legend(handles=handles, labels=new_labels, bbox_to_anchor=(1.05, 1), loc='upper left')
fig1.tight_layout()
plt.show()

# --- Print Final Generation Conditions with formatted names ---
# Create a temporary Series with formatted index for printing
formatted_final_generation_conditions_print = final_generation_conditions_values.copy()
# Apply formatting to the index for printing
formatted_final_generation_conditions_print.index = [format_condition_name(idx) for idx in final_generation_conditions_values.index]

print("Final Generation Conditions:")
print(formatted_final_generation_conditions_print.sort_values(ascending=False).to_string())

In [ ]:
fig_actions, ax_actions = plt.subplots(figsize=(15, 6)) # Use distinct fig/ax names

final_generation_actions_df = df_actions['generation'].max()

# Get the actions for the final generation, sorted by their 'value'
final_generation_actions_values = df_actions[df_actions['generation'] == final_generation_actions_df].iloc[0].drop('generation')

# Order actions based on their values in the final generation, from most to least
ordered_actions = final_generation_actions_values.sort_values(ascending=False).index.tolist()

# Convert 'action' column to a categorical type with the desired order
df_actions_melted['action'] = pd.Categorical(df_actions_melted['action'], categories=ordered_actions, ordered=True)

# Sort the DataFrame by the 'action' column to ensure correct plotting order for legend
df_actions_melted_sorted = df_actions_melted.sort_values(by='action')

sns.lineplot(data=df_actions_melted_sorted, x='generation', y='value', hue='action', ax=ax_actions)
ax_actions.set_title("Action Types Over Generations")
ax_actions.set_xlabel("Generation")
ax_actions.set_ylabel("Value") # Added y-label for clarity

# Get current legend handles and labels
handles_act, labels_act = ax_actions.get_legend_handles_labels()

# Create new labels with final generation values
new_labels_act = []
for label in labels_act:
    # 'label' here is expected to be a plain string action name
    value_at_final = final_generation_actions_values.get(label, 'N/A')
    
    # Check if value_at_final is 'N/A' (a string) before trying to format it as a number
    if value_at_final == 'N/A':
        new_labels_act.append(f"{label} ({value_at_final})")
    else:
        new_labels_act.append(f"{label} ({value_at_final:.0f})")

# Update the legend with new labels and correct position
ax_actions.legend(handles=handles_act, labels=new_labels_act, bbox_to_anchor=(1.05, 1), loc='upper left')
fig_actions.tight_layout()
plt.show()

# Print final generation actions
print("Final Generation Actions:")
print(final_generation_actions_values.sort_values(ascending=False).to_string())

In [ ]:
plt.figure(figsize=(15, 6))
sns.lineplot(data=df_performance_melted, x='generation', y='value', hue='metric')
plt.title("Performance Metrics Over Generations")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
final_generation = df_performance['generation'].max()
final_generation_performance = df_performance[df_performance['generation'] == final_generation].iloc[0].drop('generation')
print("Final Generation Performance:")
print(final_generation_performance.sort_values(ascending=False).to_string())